# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

In [1]:
import  magpie              as     magpie
import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
import  os



#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_path(obj,prefix,filetype='fits'):
    if   filetype == 'fits':
        return f"data/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"
    elif filetype == 'png':
        return f"png/{obj['field']}/{obj['field']}_{str(obj['id']).zfill(5)}.{prefix}.{filetype}"

In [ ]:
import magpie.montecarlo

def spatial_remap(obj,map,weight_map=False):
    pixel_length = (np.deg2rad(0.04/3600) * Planck18.angular_diameter_distance(obj['redshift']).to(u.kpc)).value

    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-25*pixel_length,25*pixel_length,50,
                -25*pixel_length,25*pixel_length,50)
    b2r.setup_polar_lin(0., 25*pixel_length, 12, 10, center=[0., 0.])
    b2r.get_weights() 
    remap = b2r.remap(map)

    if weight_map == True:
        err = 1/np.sum(remap,axis=0)**0.5
        return err
    else:
        r     = np.linspace(b2r.redges[0], b2r.redges[-1],12)
        map_r = np.average(remap,axis=0)
        map_r_std = np.std(remap,axis=0)
        return r,map_r,map_r_std
    

def radial_profile(obj,linemap,weight):
        linemap = linemap.data; weight = weight.data
        r, linemap_r, linemap_r_std = spatial_remap(obj,np.where(linemap>0,linemap,0))
        linemap_r_err = spatial_remap(obj,np.where(linemap>0,weight,0),weight_map=True)
        linemap_r_err = (linemap_r_err**2 + linemap_r_std**2)**0.5
        return r, linemap_r, linemap_r_err 


#%%capture
obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')

def gen_table(hdu,obj):
    path = file_path(obj=obj,prefix='extracted')
    with fits.open(path,mode='update') as hdu:
        
        r, ha_r, ha_r_err = radial_profile(obj,hdu[4],hdu[5])
        r, hb_r, hb_r_err = radial_profile(obj,hdu[11],hdu[7])
        r, hb_r, hb_r_err = radial_profile(obj,hdu[6],hdu[7])
        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        cols = [
            fits.Column(name='DISTANCE [kpc]', format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]', format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]', format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM', format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR', format='E', array=balmer_r_err),
        ]
        return fits.BinTableHDU.from_columns(cols, name='PIX_RAD_PROFILE')

for obj in tqdm(obj_lis):
    path = file_path(obj=obj,prefix='extracted')
    with fits.open(path,mode='update') as hdu:
        update = True
        if update == False: continue
        new_table = gen_table(hdu,obj)
        if len(hdu)>13:
            hdu[13] = new_table
            hdu.flush()
            os.system('clear')
        else:
            hdu.append(new_table)
            hdu.flush()
            os.system('clear')


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
%matplotlib inline
obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')

def plot_balmer_decrem(obj):
    try:
        path = file_path(obj=obj,prefix='extracted')
        with fits.open(path) as hdu:            
            ax = plt.figure(figsize=(8,10));i=1

            for index in [3,2,4,11]:
                ax.add_subplot(int(f'32{i}'));i+=1
                plt.imshow(hdu[index].data,#np.where(seg,hdu[index].data,0),
                            norm=colors.Normalize(vmin=0),
                            origin='lower',
                            cmap = 'plasma_r')
                plt.title(hdu[index].name)
                if index != 2: plt.colorbar()
            
            r,ha_r,ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err = np.vstack(hdu[13].data).transpose()            
            ax.add_subplot(325)
            plt.errorbar(r,ha_r,yerr = ha_r_err,fmt='bo:',label='Ha')
            plt.errorbar(r,hb_r,yerr = hb_r_err,fmt='go:',label='Hb')
            plt.xlabel('distance [kpc]'); plt.ylabel('flux [1e-7 erg/s/cm2]')
            plt.yscale('log');plt.grid()
            
            ax.add_subplot(326)
            plt.errorbar(r,balmer_r,yerr = balmer_r_err,fmt='ro:')
            plt.xlabel('distance [kpc]');plt.ylabel('Ha/Hb')
            plt.xlim(0,5);plt.ylim(-5,10);plt.grid()
            plt.savefig(f"radial_balmer_decrem/{obj['field']}-{obj['id']}.png")
            plt.close('all')
            return f"{obj['field']}-{obj['id']}saved"
        

    except Exception as e:
            return f"! {obj['field']}-{obj['id']} failed, error{e}"

def cat_process(obj_lis,max_threads=8):
        results = []
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(plot_balmer_decrem,obj):obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results

if __name__ == '__main__':
    use('Agg')
    obj_lis = Table.read('spectra-fitting_selected_psfmatched.fits')
    results = cat_process(obj_lis,max_threads=7)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)
